<a href="https://colab.research.google.com/github/RicardoVilla0/MachineLearning/blob/main/Unidad%203/proyecto/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Importamos librerias necesarias para Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve


In [2]:
prestamos_df = pd.read_csv('prestamos_ok.csv')
prestamos_df.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,issue_year,repaid,loan_term_year,purpose_code,home_ownership_code,grade_code,addr_state_code
0,0,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,Cash,N,NaN,2011,1,3,11,4,2,2
1,1,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,Cash,N,NaN,2011,1,3,9,4,2,0
2,2,3000,3000,3000.0,36 months,18.64,109.43,E,E1,MKC Accounting,...,Cash,N,NaN,2011,1,3,0,4,4,0
3,3,5600,5600,5600.0,60 months,21.28,152.39,F,F2,NaN,...,Cash,N,NaN,2011,0,5,11,3,5,0
4,4,5375,5375,5350.0,60 months,12.69,121.45,B,B5,Starbucks,...,Cash,N,NaN,2011,0,5,9,4,1,5


In [3]:
X = prestamos_df[['loan_amnt', 'int_rate', 'annual_inc',
                  'dti', 'purpose_code','grade_code']]

# Variable objetivo o variable a predecir
y = prestamos_df["repaid"]

In [4]:
# Dividimos el dataFrame
# stratify es para que mantenga la misma proporción de clases en ambos conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.3, stratify=y )
# verificamos la cantidad de registros asignados al dataframe de entrenamiento
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13935, 6), (5973, 6), (13935,), (5973,))

In [5]:
# Create Regressor with default properties
rfc1 = RandomForestClassifier(random_state=23)

# Fit estimator and display score
rfc1 = rfc1.fit(X_train, y_train)

# Precisión del modelo en la fase de entrenamiento
print("Precision del clasificador en fase de entrenamiento", rfc1.score(X_train, y_train) )

Precision del clasificador en fase de entrenamiento 1.0


In [6]:
# Realizar una prediccion con los datos de prueba
y_pred = rfc1.predict(X_test)

# Crear un informe de texto que muestre las principales métricas de clasificación.
print("\nReporte del clasificador Random Forest sin balanceo de clases : \n",
      classification_report(y_test, y_pred, target_names=["No Pagado", "Pagado"]))

print(f'\nMatriz de Confusion Random Forest sin balanceo de clases:\n', confusion_matrix(y_test, y_pred ))


Reporte del clasificador Random Forest sin balanceo de clases : 
               precision    recall  f1-score   support

   No Pagado       0.37      0.04      0.08       883
      Pagado       0.86      0.99      0.92      5090

    accuracy                           0.85      5973
   macro avg       0.61      0.52      0.50      5973
weighted avg       0.78      0.85      0.79      5973


Matriz de Confusion Random Forest sin balanceo de clases:
 [[  39  844]
 [  66 5024]]


In [8]:
# Optimización de Random Forest priorizando detección de impagos

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score, classification_report, confusion_matrix

# Definición del modelo base con balanceo de clases
# El parámetro class_weight='balanced' ajusta el peso inversamente proporcional
# a la frecuencia de cada clase, ayudando a tratar el desbalance.

modelo_base = RandomForestClassifier(
    class_weight='balanced',
    random_state=23
)

# Definición de la cuadrícula de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Definición del criterio de evaluación
# Se usará el recall de la clase "No Pagado" (pos_label=0)
# Esto hace que el modelo busque detectar el mayor número posible de impagos.

scorer = make_scorer(recall_score, pos_label=0)

# Configuración del GridSearchCV
grid_search = GridSearchCV(
    estimator=modelo_base,
    param_grid=param_grid,
    scoring=scorer,       # prioriza el recall de la clase No Pagado
    cv=5,                 # validación cruzada con 5 particiones
    n_jobs=-1,            # usa todos los núcleos disponibles
    verbose=2
)

# Entrenamiento del modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

print("\nBúsqueda finalizada.")
print(f"Mejores hiperparámetros encontrados:\n{grid_search.best_params_}")

# Evaluación del mejor modelo
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

print("\nReporte de Clasificación (modelo optimizado para detectar impagos):\n")
print(classification_report(y_test, y_pred))

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

Fitting 5 folds for each of 162 candidates, totalling 810 fits

Búsqueda finalizada.
Mejores hiperparámetros encontrados:
{'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}

Reporte de Clasificación (modelo optimizado para detectar impagos):

              precision    recall  f1-score   support

           0       0.22      0.66      0.33       883
           1       0.91      0.58      0.71      5090

    accuracy                           0.59      5973
   macro avg       0.56      0.62      0.52      5973
weighted avg       0.81      0.59      0.65      5973

Matriz de Confusión:
[[ 585  298]
 [2125 2965]]
